In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import optuna
import joblib
import warnings

/home/manankhatri/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df= pd.read_csv("../dataset.csv")
df.head(3)

,Year,Month,Day,SurfacePressure,MaxTemperature,MinTemperature,DewPoint,WetBulbTemp,RelHumidity,WindSpeed10,Precipitation,HEATWAVE
0,1990,1,1,98.91,22.18,4.40,-2.97,4.54,38.50,3.43,0.0,0
1,1990,1,2,98.99,21.76,5.84,-3.17,4.56,36.75,2.73,0.0,0
2,1990,1,3,99.12,23.00,5.98,-5.08,4.33,30.00,1.64,0.0,0


In [3]:
def preprocess_data(df):
    print("Preprocessing data.")

    for i in range(1, 8):
        df[f"MaxTemperature_lag_{i}"] = df["MaxTemperature"].shift(i)
        df[f"MinTemperature_lag_{i}"] = df["MinTemperature"].shift(i)
        df[f"RelHumidity_lag_{i}"] = df["RelHumidity"].shift(i)
        df[f"WindSpeed10_lag_{i}"] = df["WindSpeed10"].shift(i)
        df[f"SurfacePressure_lag_{i}"] = df["SurfacePressure"].shift(i)
        df[f"DewPoint_lag_{i}"] = df["DewPoint"].shift(i)
        df[f"WetBulbTemp_lag_{i}"] = df["WetBulbTemp"].shift(i)
        df[f"Precipitation_lag_{i}"] = df["Precipitation"].shift(i)

    for col in ['MaxTemperature', 'MinTemperature', 'RelHumidity', 'WindSpeed10', 'SurfacePressure', 'DewPoint', 'WetBulbTemp', 'Precipitation']:
        df[f"{col}_rolling_mean_7"] = df[col].rolling(window=7).mean()
    
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['Day_sin'] = np.sin(2 * np.pi * df['Day'] / 31)
    df['Day_cos'] = np.cos(2 * np.pi * df['Day'] / 31)
    
    df.dropna(inplace=True)
    return df

In [4]:
df= preprocess_data(df)

Preprocessing data.


In [5]:
X= df.drop(columns= ["HEATWAVE", "Year", "Month", "Day"])
y= df["HEATWAVE"]

In [6]:
tscv= TimeSeriesSplit(n_splits= 5)

scaler= StandardScaler()
X_scaled= scaler.fit_transform(X)

In [7]:
def objective(trial):
    params= {
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
    }

    model= XGBClassifier(**params, random_state= 42, verbose= True)
    scores= []

    print("Starting cross-validation.")

    for train_index, test_index in tscv.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
        y_pred = model.predict_proba(X_test)[:, 1]
        score = roc_auc_score(y_test, y_pred)
        scores.append(score)
    
    return np.mean(scores)

In [8]:
print("Running Optuna for Hyperparameter Tuning")

warnings.filterwarnings(action= "ignore")
study= optuna.create_study(direction= "maximize")
study.optimize(objective, n_trials= 100, show_progress_bar= True)

[I 2024-08-19 16:08:36,086] A new study created in memory with name: no-name-cfb43e4d-4fb0-4cf4-af17-fa0964d7b328


Running Optuna for Hyperparameter Tuning


  0%|          | 0/100 [00:00<?, ?it/s]

Starting cross-validation.


Best trial: 0. Best value: 0.995368:   1%|          | 1/100 [00:03<05:03,  3.06s/it]

...

[I 2024-08-19 16:19:56,993] Trial 97 finished with value: 0.9960320387485482 and parameters: {'max_depth': 14, 'learning_rate': 0.029339905675080928, 'n_estimators': 876, 'gamma': 3.648263103988663, 'subsample': 0.5454965372630414, 'colsample_bytree': 0.8373605682690906, 'colsample_bylevel': 0.9009950794027077, 'colsample_bynode': 0.8781867401769561, 'min_child_weight': 10, 'scale_pos_weight': 1.200027065156039, 'reg_alpha': 9.041712513208896, 'reg_lambda': 5.565358111895414, 'max_delta_step': 7}. Best is trial 85 with value: 0.9960839276533298.
Starting cross-validation.


Best trial: 85. Best value: 0.996084:  99%|█████████▉| 99/100 [11:30<00:08,  8.87s/it]

[I 2024-08-19 16:20:06,607] Trial 98 finished with value: 0.995970273522518 and parameters: {'max_depth': 13, 'learning_rate': 0.029350924827887842, 'n_estimators': 882, 'gamma': 3.951052487856817, 'subsample': 0.5494571248087624, 'colsample_bytree': 0.837779877414561, 'colsample_bylevel': 0.9060633662985234, 'colsample_bynode': 0.8803223137101593, 'min_child_weight': 9, 'scale_pos_weight': 2.4238859074478363, 'reg_alpha': 8.862209346588605, 'reg_lambda': 4.186224300728302, 'max_delta_step': 8}. Best is trial 85 with value: 0.9960839276533298.
Starting cross-validation.


Best trial: 85. Best value: 0.996084: 100%|██████████| 100/100 [11:38<00:00,  6.98s/it]

[I 2024-08-19 16:20:14,565] Trial 99 finished with value: 0.9958274390935804 and parameters: {'max_depth': 14, 'learning_rate': 0.042098662580154704, 'n_estimators': 825, 'gamma': 4.10004769071952, 'subsample': 0.5088324236712807, 'colsample_bytree': 0.8809999252090934, 'colsample_bylevel': 0.9363619768230396, 'colsample_bynode': 0.9019386965279707, 'min_child_weight': 2, 'scale_pos_weight': 1.1704827730945753, 'reg_alpha': 8.538250958360004, 'reg_lambda': 2.0705092825670794, 'max_delta_step': 9}. Best is trial 85 with value: 0.9960839276533298.


In [9]:
best_params= study.best_params
print("Best hyperparameters: ", best_params)

Best hyperparameters:  {'max_depth': 15, 'learning_rate': 0.08561001804950044, 'n_estimators': 710, 'gamma': 4.3130810261428385, 'subsample': 0.5912307921737588, 'colsample_bytree': 0.827648299944596, 'colsample_bylevel': 0.8645044643016258, 'colsample_bynode': 0.8948538181909136, 'min_child_weight': 10, 'scale_pos_weight': 1.1734695188379878, 'reg_alpha': 7.277321059668214, 'reg_lambda': 5.126906713495569, 'max_delta_step': 5}


In [10]:
final_model= XGBClassifier(**best_params, random_state= 42, verbose= True)
final_model.fit(X_scaled, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8645044643016258,
              colsample_bynode=0.8948538181909136,
              colsample_bytree=0.827648299944596, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=4.3130810261428385,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08561001804950044,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=5, max_depth=15, max_leaves=None,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=710, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [11]:
y_pred = final_model.predict(X_scaled)
y_proba = final_model.predict_proba(X_scaled)[:, 1]

In [12]:
y_proba.shape

(12603,)

In [13]:
auc_roc = roc_auc_score(y, y_proba)
print(f"AUC-ROC score: {auc_roc}")
print(classification_report(y, y_pred))

AUC-ROC score: 0.9982094047098619
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     11919
           1       0.88      0.92      0.90       684

    accuracy                           0.99     12603
   macro avg       0.94      0.96      0.95     12603
weighted avg       0.99      0.99      0.99     12603



In [32]:
joblib.dump(final_model, 'MODEL.pkl')
joblib.dump(scaler, 'Scaler.pkl')

['Scaler.pkl']

In [8]:
def validate_model(model, X, y):
    print("Validating the model...")
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]
    
    auc_roc = roc_auc_score(y, y_proba)
    print(f"AUC-ROC score: {auc_roc}")
    print(classification_report(y, y_pred))

In [9]:
model = joblib.load('MODEL.pkl')
scaler = joblib.load('scaler.pkl')

In [10]:
validate_model(model, X_scaled, y)

Validating the model...
AUC-ROC score: 0.9982094047098619
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     11919
           1       0.88      0.92      0.90       684

    accuracy                           0.99     12603
   macro avg       0.94      0.96      0.95     12603
weighted avg       0.99      0.99      0.99     12603



In [11]:
def predict_heatwave(model, recent_data):
    print("Predicting heatwave based on the last 10 days of data...")
    X_recent = preprocess_data(recent_data).tail(1)
    X_recent_scaled = scaler.transform(X_recent.drop(columns=['HEATWAVE', 'Year', 'Month', 'Day']))
    
    prediction = model.predict(X_recent_scaled)
    probability = model.predict_proba(X_recent_scaled)[:, 1]
    
    return prediction[0], probability[0]

In [12]:
recent_data = pd.read_csv("../subsets/file_159.csv")
heatwave_pred, heatwave_proba = predict_heatwave(model, recent_data)
print(f"Heatwave Prediction: {heatwave_pred}, Probability: {heatwave_proba}")

Predicting heatwave based on the last 10 days of data...
Preprocessing data.
Heatwave Prediction: 1, Probability: 0.7116962671279907


In [13]:
recent_data

,Year,Month,Day,SurfacePressure,MaxTemperature,MinTemperature,DewPoint,WetBulbTemp,RelHumidity,WindSpeed10,...,RelHumidity_rolling_mean_7,WindSpeed10_rolling_mean_7,SurfacePressure_rolling_mean_7,DewPoint_rolling_mean_7,WetBulbTemp_rolling_mean_7,Precipitation_rolling_mean_7,Month_sin,Month_cos,Day_sin,Day_cos
7,1994,5,17,97.61,43.47,25.28,3.95,19.28,16.56,2.91,...,20.090000,3.762857,97.797143,5.134286,18.980000,0.700000,0.5,-0.866025,-0.299363,-0.954139
8,1994,5,18,97.52,44.69,26.61,5.82,20.99,16.56,2.84,...,20.161429,3.644286,97.762857,5.897143,19.704286,0.497143,0.5,-0.866025,-0.485302,-0.874347
9,1994,5,19,97.47,45.94,28.36,6.98,22.47,15.81,2.45,...,20.384286,3.614286,97.725714,6.744286,20.400000,0.497143,0.5,-0.866025,-0.651372,-0.758758
